# Description
This example is a simple demonstration of how to run a Sesam Core fatigue hotspot example using OneWorkflow locally and in the cloud.

In [ ]:
from dnv.oneworkflow.utils.workunit_extension import *
from dnv.oneworkflow.utils.starter import *
from pathlib import Path
import os
await install_workflow_runtime(repository = PackageManager.Repository.DEV)

In [ ]:
from dnv.oneworkflow.utils.workunit_extension import *
from dnv.oneworkflow.utils.starter import *
from dnv.oneworkflow import OneWorkflowClient
from pathlib import Path
import os
oneWorkflowTMPFolder = r'c:\OneWorkflowTMP' #due to possible issues with long file paths we prefer to have this folder at the root
if not os.path.exists(oneWorkflowTMPFolder):
    try:
        print("Trying to create tmp folder for one workflow local execution")
        os.mkdir(oneWorkflowTMPFolder)
    except:
        print("did not manage to create tmp folder for local execution. Check that you have privileges to create it or try to manually create it from the coomand line.")

# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath and in the folder names CommonFiles
root_folder = os.getcwd()
workspacePath = str(Path(root_folder, 'Workspace'))
workspaceId = "SesamCoreExample"
cloudRun = False
#If running locally the code below will also start the local workflow host.
workflow_client = one_workflow_client(workspace_id = workspaceId, workspace_path = workspacePath, cloud_run = cloudRun,
                                      local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder, platform=Platform.WINDOWS, max_cores=5,auto_deploy_option = AutoDeployOption.DEV)
workflow_client.start_workflow_runtime_service()
if (cloudRun):
    workflow_client.login()
upload_success = workflow_client.upload_common_files(FileOptions(max_size="524MB",patterns=["**/*.*"], overwrite=True))

# max number of threads used when running locally


In [ ]:
import pandas as pd
import os
from dnv.oneworkflow import  ParallelWork
from dnv.onecompute.flowmodel import WorkUnit
from dnv.sesam.sesam_core_command import *
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
import json
import shutil
from dnv.oneworkflow.utils.workunit_extension import with_shared_files_copied_to_loadcase

# we must delete existing results locally before generating new results
local__result_path = Path(workspacePath, workflow_client.results_directory)
if os.path.isdir(local__result_path):
    shutil.rmtree(local__result_path) 

parallel_work = ParallelWork()
for index in range(1,14): # iterating over two simple cases, they now will do the same analysis
    loadcase_folder_name = f"LoadCase{index}"
    result_folder_lc = os.path.join(workflow_client.results_directory, loadcase_folder_name)
    python_copy_command = PythonCommand(
        directory=workflow_client.common_directory)
    core_command = SesamCoreCommand(command = "fatigue",input_file_name= "Specimen1_input.json", options = "-hs")
    cmd = CompositeExecutableCommand([core_command], result_folder_lc)
    work_unit = (WorkUnit(cmd, loadcase_folder_name)
                 .output_directory(result_folder_lc, include_files=["**/*.txt",  "**/*.MLG", "**/*.lis", "**/*.sin", "*.log"])
                .with_shared_files_copied_to_loadcase(workflow_client.common_directory, ["**/*.py"])
            )
    parallel_work.WorkItems.append(work_unit)
downloadOptions = FileOptions(max_size="10MB",patterns=["**/*.txt", "**/*.lis", "**/*.MLG"])
job = workflow_client.create_job(parallel_work)
#for debugging
#job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
await run_workflow_async(job, workflow_client, downloadOptions)



### CLose client -must be done before a new job can be started in a different notebook
Will remove all job and blob folders.

In [ ]:
workflow_client.stop_workflow_runtime_service()